# Tratamento dos Dados Coletados

O tratamento dos dados foi feito por meio das bibliotecas **Pandas** e **Unicode**, que foram utilizadas respectivamente para manipulação de Datasets e unificação de strings

In [169]:
import pandas as pd
from unidecode import unidecode
import numpy as np
pd.set_option('display.expand_frame_repr', False)
pd.set_option('max_colwidth', -1)

C:\Users\guide\AppData\Local\Temp\ipykernel_10468\505288178.py:5: FutureWarning: Passing a negative integer is deprecated in version 1.0 and will not be supported in future version. Instead, use None to not limit the column width.
  pd.set_option('max_colwidth', -1)


Primeiramente é lido o Dataset criado anteriormente, e então agrupado por titulos, para que seja criado um ID único para cada vídeo.

In [170]:
df = pd.read_csv('youtube.csv')
df['id_no'] = df.groupby(['titulo']).ngroup()
list(set(df['proximo_video_titulo'].unique().tolist())-set(df['titulo'].unique().tolist()))
df = df[['id_no', 'titulo', 'proximo_video_titulo', 'visualizacoes', 'curtidas', 'canal', 'data', 'url']]
df = df.sort_values(by=['id_no'])
df.to_csv('youtube.csv',index=False)


Então, é contado as linhas duplicadas. Uma linha duplicada significa que um mesmo caminho foi percorrido duas vezes, para cada vez que um caminho é percorrido o peso das arestas entre dois vídeos é incrementado em 1.
E então estes dados são salvos no mesmo Dataset com as novas colunas de ID, ID do proximo nó (video) e o peso das arestas (conexão entre dois vídeos).

In [171]:
df['peso'] = df.groupby(['titulo', 'proximo_video_titulo'])['titulo'].transform('count')

df = df[['id_no', 'peso', 'titulo', 'proximo_video_titulo', 'visualizacoes', 'curtidas', 'canal', 'data', 'url']]
df['peso'] = df['peso'].astype(int)
df.to_csv('youtube.csv',index=False)


Após a criação dos IDs únicos, cada vídeo recomendado tem seu ID unico referente aos IDs criados anteriormente, ou seja, a coluna id_proximo_no é preenchida.

In [172]:
df = pd.read_csv('youtube.csv')
df['id_no'] = df.groupby(['titulo']).ngroup()

for a in df['titulo'].unique().tolist(): # para cada titulo unico
    print("Titulo a: ", a)
    for b in df['proximo_video_titulo'].tolist():  # para cada proximo_video_titulo da lista de titulos
        print("Proximo video b: ", b)
        if b != '-1': # se o proximo_video_titulo for diferente de -1 (ultimo video da lista que nao possui proximo video)
            if a == b:
                print("a == b", a, b)
                # adiciona o id_no do titulo na coluna id_prox_no referente ao proximo_video_titulo
                df.loc[df['proximo_video_titulo'] == unidecode(b), 'id_prox_no'] = df.loc[df['titulo'] == unidecode(a), 'id_no'].values[0]
        else: 
            df.loc[df['proximo_video_titulo'] == unidecode(b), 'id_prox_no'] = df.loc[df['titulo'] == unidecode(a), 'id_no'].values[0] # se o proximo_video_titulo for igual a -1, adiciona o id_no do titulo na coluna id_prox_no

df = df.astype({'id_prox_no':'int'})
df

Titulo a:  A Melhor Historia Brasileira de Aviacao | EP. 1072
Proximo video b:  Boeing 777 - o aviao FAVORITO do Lito | EP. 777
Proximo video b:  Voos United 585 e UsAir 427 - Defeito FATAL | EP. 1033
Proximo video b:  Bolsonaro (PL) pergunta para Padre Kelmon (PTB) sobre Lei Rouanet #DebateNaGlobo
Proximo video b:  Padre Kelmon (PTB) pergunta para Ciro Gomes (PDT) sobre educacao #DebateNaGlobo
Proximo video b:  Simone Tebet (MDB) pergunta para Soraya Thronicke (Uniao Brasil) sobre educacao #DebateNaGlobo
Proximo video b:  Padre Kelmon (PTB) pergunta para Lula (PT) sobre corrupcao #DebateNaGlobo
Proximo video b:  Padre de festa junina, extrema-uncao e catequese: o embate entre Soraya Thronicke e Padre Kelmon
Proximo video b:  Felipe D'Avila (Novo) pergunta para Lula (PT) sobre cotas raciais #DebateNaGlobo
Proximo video b:  Soraya Thronicke (Uniao Brasil) pergunta para Padre Kelmon (PTB) sobre educacao #DebateNaGlobo
Proximo video b:  Ciro Gomes (PDT) pergunta para Bolsonaro (PL) sobre 

,id_no,peso,titulo,proximo_video_titulo,visualizacoes,curtidas,canal,data,url,id_prox_no
0,0,1,A Melhor Historia Brasileira de Aviacao | EP. 1072,Boeing 777 - o aviao FAVORITO do Lito | EP. 777,366432,68000.0,Avioes e Musicas,27 de nov. de 2022,https://www.youtube.com/watch?v=A8h9ZfWaWCo,1
1,1,1,Boeing 777 - o aviao FAVORITO do Lito | EP. 777,Voos United 585 e UsAir 427 - Defeito FATAL | EP. 1033,2549107,103000.0,Avioes e Musicas,13 de jun. de 2021,https://www.youtube.com/watch?v=P_WRAUmsoHk,40
2,2,1,Bolsonaro (PL) pergunta para Felipe D'Avila (Novo) sobre governos de esquerda #DebateNaGlobo,Bolsonaro (PL) pergunta para Padre Kelmon (PTB) sobre Lei Rouanet #DebateNaGlobo,234918,4600.0,g1,30 de set. de 2022,https://www.youtube.com/watch?v=enshEm19iTI,4
3,3,1,Bolsonaro (PL) pergunta para Felipe D'Avila (Novo) sobre relacao com o Congresso #DebateNaGlobo,Padre Kelmon (PTB) pergunta para Ciro Gomes (PDT) sobre educacao #DebateNaGlobo,248353,3700.0,g1,30 de set. de 2022,https://www.youtube.com/watch?v=vCs_VR6NoAY,25
4,4,1,Bolsonaro (PL) pergunta para Padre Kelmon (PTB) sobre Lei Rouanet #DebateNaGlobo,Simone Tebet (MDB) pergunta para Soraya Thronicke (Uniao Brasil) sobre educacao #DebateNaGlobo,568522,11000.0,g1,30 de set. de 2022,https://www.youtube.com/watch?v=LIUq9sLrpfM,34
...,...,...,...,...,...,...,...,...,...,...
90,37,7,Soraya Thronicke (Uniao Brasil) pergunta para Padre Kelmon (PTB) sobre educacao #DebateNaGlobo,Lula (PT) pergunta para Ciro Gomes (PDT) sobre cultura #DebateNaGlobo,423484,6000.0,g1,30 de set. de 2022,https://www.youtube.com/watch?v=0rlYVPffxXQ,21
91,37,4,Soraya Thronicke (Uniao Brasil) pergunta para Padre Kelmon (PTB) sobre educacao #DebateNaGlobo,Lula (PT) pergunta para Ciro Gomes (PDT) sobre agricultura #DebateNaGlobo,423484,6000.0,g1,30 de set. de 2022,https://www.youtube.com/watch?v=0rlYVPffxXQ&t=4s,20
92,38,1,"Tenerife, O MAIOR acidente aereo de todos os TEMPOS EP. 156",-1,7172977,347000.0,Avioes e Musicas,2 de abr. de 2017,https://www.youtube.com/watch?v=t3FLv5nZtjg,40
93,39,1,Thronicke (Uniao Brasil) responde pergunta sobre imposto unico; Kelmon (PTB) comenta | CNN BRASIL,Bolsonaro (PL) responde pergunta sobre violencia politica; Ciro Gomes (PDT) comenta | CNN BRASIL,17827,321000.0,CNN Brasil,24 de set. de 2022,https://www.youtube.com/watch?v=HhT2XSPP2eE,5


In [173]:
df = df[['id_no', 'id_prox_no', 'peso', 'titulo', 'proximo_video_titulo', 'visualizacoes', 'curtidas', 'canal', 'data', 'url']]
dfsorted = df.sort_values(by=['id_no', 'id_prox_no'])
dfsorted.drop_duplicates(subset=['id_no', 'id_prox_no'], keep='first', inplace=True)
dfsorted.to_csv('youtube.csv',index=False)
